In [1]:
import requests
import json
from random import randint
from tqdm import trange
import os
import pandas as pd
import re
from groq import Groq
import random

In [3]:
def get_output(message):
    linshi=random.randint(0,4)
    client = Groq(
        api_key=key_pool[linshi],
    )
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": message,
            }
        ],
        model="llama3-70b-8192",
        temperature=0
    )
    return chat_completion.choices[0].message.content

In [4]:
def get_bias_output(ans,fact):
    bias=''
    if ans=='A':
        bias='Among all the questions, 30% of them have the answer is no and 70% have the answer is yes.'
    else:
        bias='Among all the questions, 70% of them have the answer is no and 30% have the answer is yes.'
    template="""Here is a prompt that may be biased by cognitive bias. Rewrite it such that a reviewer is not biased.

Prompt:
Please answer the question.
%s

Question: %s

There is no need to output which sentences that have deleted or changed.
"""%(bias,fact)
    return template

In [5]:
f=open("../data/test.jsonl")
for i in trange(500):
    now=json.loads(f.readline())
    if i<=220:
        continue
    temp1=get_bias_output(str(now['ans']),now['question'])
    ans1=get_output(temp1)
    #print(ans1)
    ans1=re.findall(r"([\S|\s]*)Please([\S|\s]*)",ans1, re.S|re.I)[0][1]
    ans1='Please'+ans1
    ans1=re.findall(r"([\S|\s]*)\?([\S|\s]*)",ans1, re.S|re.I)[0][0]
    ans1=ans1+'?'
    ans2=get_output(ans1+'\nA.no   B.yes\nOutput format:\nAns:  (Only one option can be selected)')
    with open("self_help.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'bias':ans2,'ans':str(now['ans']),'prompt':ans1}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')



100%|██████████| 500/500 [21:05<00:00,  2.53s/it]
